# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
CLIENT_PORT = 62012
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", CLIENT_PORT)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Thu 2023-01-19 23:01:46.657559 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE
Thu 2023-01-19 23:01:46.658222 [INFO ] - [T] Attempting to bind to 0.0.0.0:62012
Thu 2023-01-19 23:01:46.658251 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Thu 2023-01-19 23:01:46.658844 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
Thu 2023-01-19 23:01:46.695602 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7f48a0af6990>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
CRESET    = '\33[0m'
CBOLD     = '\33[1m'
CITALIC   = '\33[3m'
CURL      = '\33[4m'
CBLINK    = '\33[5m'
CBLINK2   = '\33[6m'
CSELECTED = '\33[7m'

CBLACK  = '\33[30m'
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'
CVIOLET = '\33[35m'
CBEIGE  = '\33[36m'
CWHITE  = '\33[37m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    else:
        color = CVIOLET
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7f48a0ba4f40>, <FilterInterface at 0x7f48a0ba4b80>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient1438", client_version: "0.0.666" }

In [11]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")

filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient1438", client_version: "0.0.666" } }

In [12]:
await client_fiface.command(filter_cmd)

Thu 2023-01-19 23:01:46.881382 [TRACE] - [F<-A,C] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient1438", client_version: "0.0.666" } }


In [13]:
# XXX get endpoint programatically from client
client_endpoint = EndpointW(f"127.0.0.1:{CLIENT_PORT}")
client_endpoint

Endpoint(127.0.0.1:62012)

In [14]:
response = await server_fiface.response()
response

LTI(client)5 skipped 4: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }
Thu 2023-01-19 23:01:46.904704 [TRACE] - [T<-F,C] Processing command NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }
LTI(client)6: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI(client)7: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient1438", client_version: "0.0.666" } }] }
Thu 2023-01-19 23:01:46.905036 [TRACE] - [F<-T,R] Command Accepted
Thu 2023-01-19 23:01:46.905104 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, re

Accepted

LTI(server)7 skipped 6: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62012), timeout: 5s }
Thu 2023-01-19 23:01:46.931026 [TRACE] - [T<-F,C] Processing command NewEndpoint { endpoint: Endpoint(127.0.0.1:62012), timeout: 5s }


In [15]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
new_request_action = server_notifications[0]

assert new_request_action.request_action == connect_action

In [16]:
cookie = base64.urlsafe_b64encode(bytes(new_request_action.request_action.name.encode()))
cookie

b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'

In [17]:
response_ok = ResponseCodeW("loggedin", cookie=str(cookie), server_version="0.0.666")
response_ok

ResponseCodeW { inner: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }

In [18]:
await server_fiface.command(FilterCmdW("sendresponsecode", endpoint=client_endpoint, code=response_ok))

LTI(server)8: got transport notifications: []
LTI(server)9: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-19 23:01:46.965835 [TRACE] - [F<-A,C] New command: SendResponseCode { endpoint: Endpoint(127.0.0.1:62012), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }
Thu 2023-01-19 23:01:46.966033 [TRACE] - [F<-T,R] Command Accepted


In [20]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

NewResponseCode { endpoint: Endpoint(127.0.0.1:62013), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }

In [21]:
new_response_code.response_code

ResponseCodeW { inner: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }

In [22]:
cookie_from_server = new_response_code.response_code.cookie
assert str(cookie_from_server) == str(cookie)

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
client_fiface.get_notifications()

LTI(server)11 skipped 1: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 9999s }], packets: [Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }] }
Thu 2023-01-19 23:01:47.002075 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 9999s }], packets: [Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.666" } }] }
LTI(server)12: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-19 23:01:47.002128 [TRACE] - [T<-UDP] Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'", server_version: "0.0.6

LTI(client)50: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)51: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 7, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
LTI(client)52: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-19 23:01:49.902378 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 250ms }], packets: [Request { sequence: 7, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 

LTI(client)72: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)73: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 3 }, retry_interval: 250ms }], packets: [Request { sequence: 16, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:50.307333 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 3 }, retry_interval: 250ms }], packets: [Request { sequence: 16, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
LTI

LTI(client)88: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)89: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 8 }, retry_interval: 250ms }], packets: [Request { sequence: 23, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:50.509350 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 8 }, retry_interval: 250ms }], packets: [Request { sequence: 23, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2t

LTI(client)106: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)107: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 31, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:50.711448 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 31, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb

...

Thu 2023-01-19 23:01:51.317537 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 25, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.317552 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 26, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.317559 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.317564 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 9, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 20

Thu 2023-01-19 23:01:51.519696 [TRACE] - [T<-UDP] Request { sequence: 44, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.519720 [TRACE] - [T<-UDP] Request { sequence: 45, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.519725 [TRACE] - [T<-UDP] Request { sequence: 46, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.519730 [TRACE] - [T<-UDP] Request { sequence: 47, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
LTI(client)190: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::Endpoin

Thu 2023-01-19 23:01:51.721598 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 7, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.721611 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 8, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.721617 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 19, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 2023-01-19 23:01:51.721622 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 20, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }
Thu 20

LTI(client)246: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)247: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 28 }, retry_interval: 250ms }], packets: [Request { sequence: 95, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQxNDM4'"), action: KeepAlive { latest_response_ack: 1 } }] }
Thu 2023-01-19 23:01:51.927408 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId

LTI(client)272: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Thu 2023-01-19 23:01:52.130675 [WARN ] - [F<-T,N] error sending KeepAlive during idle endpoint (Endpoint(127.0.0.1:62013)): Filter does not contain an entry for the endpoint: Endpoint(127.0.0.1:62013)
LTI(client)273: got transport notifications: []
LTI(server)338 skipped 241: received transport_cmd DropEndpoint { endpoint: Endpoint(127.0.0.1:62012) }
Thu 2023-01-19 23:01:57.040277 [INFO ] - [F<-T,N] Endpoint Endpoint(127.0.0.1:62012) timed-out. Dropping.
Thu 2023-01-19 23:01:57.040947 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:62012) }
LTI(server)339: got transport_rsp back from Transport layer: TransportRsp::Accepted
Thu 2023-01-19 23:01:57.041272 [TRACE] - [F<-T,R] Command Accepted
LTI(server)340: got transport notifications: []
